# Entrenamiento de Emergencia YOLOv8s en Google Colab

Este notebook está configurado con parámetros conservadores para asegurar la finalización del entrenamiento en un entorno con recursos limitados y un plazo ajustado.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics

In [ ]:
import os
import tarfile
from ultralytics import YOLO

# --- PARÁMETROS DE CONFIGURACIÓN DE EMERGENCIA ---
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision'

# --- Dataset (Comprimido) ---
COMPRESSED_DATASET_NAME = 'dataset_v2_yolov8_obb.tar.gz'
COMPRESSED_DATASET_PATH = os.path.join(DRIVE_PROJECT_PATH, COMPRESSED_DATASET_NAME)
DATASET_BASE_PATH = '/content/dataset_v2_yolov8_obb'
DATASET_YAML_PATH = os.path.join(DATASET_BASE_PATH, 'data.yaml')

# --- Lógica de Descompresión ---
if not os.path.exists(DATASET_BASE_PATH):
    print(f"Dataset no encontrado en {DATASET_BASE_PATH}. Descomprimiendo...")
    if not os.path.exists(COMPRESSED_DATASET_PATH):
        raise FileNotFoundError(f"El archivo {COMPRESSED_DATASET_PATH} no existe.")
    with tarfile.open(COMPRESSED_DATASET_PATH, 'r:gz') as tar_ref:
        tar_ref.extractall('/content/')
    print("Descompresión completada.")
else:
    print(f"Dataset ya encontrado en {DATASET_BASE_PATH}.")

# --- Configuración del Modelo y Entrenamiento ---
MODEL_NAME = 'yolov8s.pt'
EPOCHS = 80
RUN_NAME = 'yolov8s_obb_b8_w4_img640_ep80_colab_run_3'
BATCH_SIZE = 8
IMG_SIZE = 640
WORKERS = 4
PATIENCE = 20
CACHE = False
OPTIMIZER = 'AdamW'
COS_LR = True

print('--- CONFIGURACIÓN DE ENTRENAMIENTO ---')
print(f'Modelo: {MODEL_NAME}')
print(f'Epochs: {EPOCHS}')
print(f'Batch Size: {BATCH_SIZE}')
print(f'Image Size: {IMG_SIZE}')
print('----------------------------------------')
print('--- COMPROBACIONES DE DATASET, YAML Y MODELO ---')      
print(f"Dataset listo para usar en: {DATASET_BASE_PATH}")
print(f"Archivo YAML: {DATASET_YAML_PATH}")
print(f"Modelo base: {MODEL_NAME}")
print('----------------------------------------')

# Cargar el modelo
model = YOLO(MODEL_NAME)

# Entrenar
results = model.train(
    data=DATASET_YAML_PATH,
    project=os.path.join(DRIVE_PROJECT_PATH, 'training_results'),
    name=RUN_NAME,
    epochs=EPOCHS,
    patience=PATIENCE,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    workers=WORKERS,
    cache=CACHE,
    optimizer=OPTIMIZER,
    cos_lr=COS_LR,
    device='0',
    plots=True,
    save_json=True
)

print('--- ENTRENAMIENTO FINALIZADO ---')
print(f'Resultados guardados en: {results.save_dir}')

In [ ]:
# Cargar el mejor modelo guardado durante el entrenamiento
best_model_path = os.path.join(DRIVE_PROJECT_PATH, 'training_results', RUN_NAME, 'weights', 'best.pt')
model = YOLO(MODEL_NAME)

# Evaluar en el conjunto de validación
metrics = model.val()

print('--- MÉTRICAS DE VALIDACIÓN ---')
print(f'mAP50-95: {metrics.box.map:.4f}')
print(f'   mAP50: {metrics.box.map50:.4f}')
print(f'   mAP75: {metrics.box.map75:.4f}')